# D'abord on récupére l'API

In [1]:
import wikipediaapi

In [2]:
wiki_wiki = wikipediaapi.Wikipedia('fr')

In [3]:
wiki_html = wikipediaapi.Wikipedia(
        language='fr',
        extract_format=wikipediaapi.ExtractFormat.HTML
)

In [4]:
p_html = wiki_html.page("Liste_de_personnalités_liées_à_Paris")

In [5]:
p_html.text

'<h2><span id="Naissances_c.C3.A9l.C3.A8bres_.C3.A0_Paris"></span><span id="Naissances_célèbres_à_Paris">Naissances célèbres à Paris</span></h2>\n<p><small><i>(classement par année de naissance)</i></small>\n</p>\n<h3><span id="XVe_si.C3.A8cle"></span><span id="XVe_siècle"><abbr class="abbr" title="15ᵉ siècle"><span>XV</span><sup style="font-size:72%">e</sup></abbr>\xa0siècle</span></h3>\n<dl><dd>Guillaume Budé, humaniste, né le <time class="nowrap date-lien" datetime="1468-02-04" data-sort-value="1468-02-04">26 janvier 1468</time></dd>\n<dd>François II de Rohan, évêque, né en 1480.</dd>\n<dd>Antoine de Montbel, mémorialiste , né le 12 septembre 1487</dd></dl><h3><span id="XVIe_si.C3.A8cle"></span><span id="XVIe_siècle"><abbr class="abbr" title="16ᵉ siècle"><span>XVI</span><sup style="font-size:72%">e</sup></abbr>\xa0siècle</span></h3>\n<dl><dd>Richelieu, cardinal et homme politique, ancien Principal ministre de Louis XIII, né le <time class="nowrap date-lien" datetime="1585-09-28" dat

In [6]:
from bs4 import BeautifulSoup

In [7]:
content = BeautifulSoup(p_html.text, "lxml").text

In [8]:
content

"Naissances célèbres à Paris\n(classement par année de naissance)\n\nXVe\xa0siècle\nGuillaume Budé, humaniste, né le 26 janvier 1468\nFrançois II de Rohan, évêque, né en 1480.\nAntoine de Montbel, mémorialiste , né le 12 septembre 1487XVIe\xa0siècle\nRichelieu, cardinal et homme politique, ancien Principal ministre de Louis XIII, né le 28 septembre 1585XVIIe\xa0siècle\nNicolas Fouquet, homme d'État, surintendant des finances, né le 27 janvier 1615\nCharles Le Brun, peintre, né en 1619\nMolière, dramaturge, né le 15 janvier 1622\nMadame de Sévigné, écrivaine, née le 5 février 1626\nCharles Perrault, écrivain et conteur, né le 12 janvier 1628\nFrançois de Nesmond, évêque de Bayeux, né le 31 août 1629\nMadame de La Fayette, écrivaine, née le 18 mars 1634\nNicolas Boileau, écrivain, poète et critique, né le 1er novembre 1636\nLouvois, ministre d'État de la Guerre de Louis XIV, né le 18 janvier 1641\nMarc-Antoine Charpentier, compositeur, en 1643\nRomain Coupier de Maille, avocat à Paris.\n

# Ensuite on récupère les PER avec SPACY

In [9]:
from spacy import displacy

In [10]:
import spacy

In [11]:
nlp = spacy.load("fr_core_news_sm")

In [12]:
wikitext = nlp(content)

In [13]:
displacy.render(wikitext, style="ent", jupyter=True)

In [14]:
personnalites = []
for word in wikitext.ents:
    if(word.label_ == "PER"):
        pers = {}
        pers = word.text.lower()
        personnalites.append(pers)

In [15]:
import pandas as pd

In [16]:
pd.options.display.max_rows = 3000
pd.options.display.max_colwidth = 999

In [17]:
s = pd.Series(personnalites, name='personnalites')

In [18]:
s

0                                  guillaume budé
1                            françois ii de rohan
2                              antoine de montbel
3                                       richelieu
4                                      louis xiii
5                                 nicolas fouquet
6                                 charles le brun
7                                         molière
8                               madame de sévigné
9                                charles perrault
10                            françois de nesmond
11                           madame de la fayette
12                                nicolas boileau
13                                        louvois
14                                      louis xiv
15                                           marc
16                            antoine charpentier
17                       romain coupier de maille
18                             jean de la bruyère
19                                 jules hardouin


In [19]:
len(s)

2026

In [20]:
s = s.drop_duplicates()

In [21]:
s = s.reset_index()

In [22]:
s = s["personnalites"]

In [23]:
len(s)

886

In [27]:
s

0                                 guillaume budé
1                           françois ii de rohan
2                             antoine de montbel
3                                      richelieu
4                                     louis xiii
5                                nicolas fouquet
6                                charles le brun
7                                        molière
8                              madame de sévigné
9                               charles perrault
10                           françois de nesmond
11                          madame de la fayette
12                               nicolas boileau
13                                       louvois
14                                     louis xiv
15                                          marc
16                           antoine charpentier
17                      romain coupier de maille
18                            jean de la bruyère
19                                jules hardouin
20                  

In [30]:
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [47]:
print("""
    SELECT ?item ?itemLabel ?dateOfBirth ?dateOfDeath ?placeOfBirthLabel ?placeOfDeathLabel (GROUP_CONCAT(DISTINCT ?positionLabel;separator=", ") AS ?positions) 
    WHERE {
      SERVICE wikibase:mwapi {
          bd:serviceParam wikibase:api "EntitySearch" .
          bd:serviceParam wikibase:endpoint "www.wikidata.org" .
          bd:serviceParam mwapi:search '"""+s[0]+"""'.
          bd:serviceParam mwapi:language "fr" .
          ?item wikibase:apiOutputItem mwapi:item .
          ?num wikibase:apiOrdinal true .
      }
      ?item wdt:P31 wd:Q5.
      {?item wdt:P569 ?dateOfBirth.} UNION {MINUS {?item wdt:P569 ?dateOfBirth.}}
      {?item wdt:P570 ?dateOfDeath.} UNION {MINUS {?item wdt:P570 ?dateOfDeath.}}
      {?item wdt:P19 ?placeOfBirth.} UNION {MINUS {?item wdt:P19 ?placeOfBirth.}}
      {?item wdt:P20 ?placeOfDeath.} UNION {MINUS {?item wdt:P20 ?placeOfDeath.}}
      {?item wdt:P39 ?position.} UNION {MINUS {?item wdt:P39 ?position.}}

      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". 
                              ?item rdfs:label ?itemLabel .
                             ?placeOfBirth rdfs:label ?placeOfBirthLabel .
                             ?placeOfDeath rdfs:label ?placeOfDeathLabel .
                             ?position rdfs:label ?positionLabel .}

    } GROUP BY ?item ?itemLabel ?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel
    LIMIT 1
    """)


    SELECT ?item ?itemLabel ?dateOfBirth ?dateOfDeath ?placeOfBirthLabel ?placeOfDeathLabel (GROUP_CONCAT(DISTINCT ?positionLabel;separator=", ") AS ?positions) 
    WHERE {
      SERVICE wikibase:mwapi {
          bd:serviceParam wikibase:api "EntitySearch" .
          bd:serviceParam wikibase:endpoint "www.wikidata.org" .
          bd:serviceParam mwapi:search 'guillaume budé'.
          bd:serviceParam mwapi:language "fr" .
          ?item wikibase:apiOutputItem mwapi:item .
          ?num wikibase:apiOrdinal true .
      }
      ?item wdt:P31 wd:Q5.
      {?item wdt:P569 ?dateOfBirth.} UNION {MINUS {?item wdt:P569 ?dateOfBirth.}}
      {?item wdt:P570 ?dateOfDeath.} UNION {MINUS {?item wdt:P570 ?dateOfDeath.}}
      {?item wdt:P19 ?placeOfBirth.} UNION {MINUS {?item wdt:P19 ?placeOfBirth.}}
      {?item wdt:P20 ?placeOfDeath.} UNION {MINUS {?item wdt:P20 ?placeOfDeath.}}
      {?item wdt:P39 ?position.} UNION {MINUS {?item wdt:P39 ?position.}}

      SERVICE wikibase:label { bd:se

In [66]:
from urllib.error import HTTPError

In [67]:
import time
persos = []
for i in range(len(s)):
    print(s[i].replace("'"," "))
    try:
        sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        sparql.setQuery("""
        SELECT ?item ?itemLabel ?dateOfBirth ?dateOfDeath ?placeOfBirthLabel ?placeOfDeathLabel (GROUP_CONCAT(DISTINCT ?positionLabel;separator=", ") AS ?positions) 
        WHERE {
          SERVICE wikibase:mwapi {
              bd:serviceParam wikibase:api "EntitySearch" .
              bd:serviceParam wikibase:endpoint "www.wikidata.org" .
              bd:serviceParam mwapi:search '"""+s[i].replace("'"," ")+"""'.
              bd:serviceParam mwapi:language "fr" .
              ?item wikibase:apiOutputItem mwapi:item .
              ?num wikibase:apiOrdinal true .
          }
          ?item wdt:P31 wd:Q5.
          {?item wdt:P569 ?dateOfBirth.} UNION {MINUS {?item wdt:P569 ?dateOfBirth.}}
          {?item wdt:P570 ?dateOfDeath.} UNION {MINUS {?item wdt:P570 ?dateOfDeath.}}
          {?item wdt:P19 ?placeOfBirth.} UNION {MINUS {?item wdt:P19 ?placeOfBirth.}}
          {?item wdt:P20 ?placeOfDeath.} UNION {MINUS {?item wdt:P20 ?placeOfDeath.}}
          {?item wdt:P39 ?position.} UNION {MINUS {?item wdt:P39 ?position.}}

          SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],fr". 
                                  ?item rdfs:label ?itemLabel .
                                 ?placeOfBirth rdfs:label ?placeOfBirthLabel .
                                 ?placeOfDeath rdfs:label ?placeOfDeathLabel .
                                 ?position rdfs:label ?positionLabel .}

        } GROUP BY ?item ?itemLabel ?dateOfBirth ?placeOfBirthLabel ?dateOfDeath ?placeOfDeathLabel
        LIMIT 1
        """)

        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        for r in results['results']['bindings']:
            perso = {}
            if("itemLabel" in r):
                perso['nom'] = r['itemLabel']['value']
            if("dateOfBirth" in r):
                perso['dateOfBirth'] = r['dateOfBirth']['value']
            if("placeOfBirthLabel" in r):
                perso['placeOfBirthLabel'] = r['placeOfBirthLabel']['value']
            if("dateOfDeath" in r):
                perso['dateOfDeath'] = r['dateOfDeath']['value']
            if("placeOfDeathLabel" in r):
                perso['placeOfDeathLabel'] = r['placeOfDeathLabel']['value']
            if("positions" in r):
                perso["positions"] = r["positions"]["value"]
            persos.append(perso)
    except HTTPError as err:
        print("except - try again")
        i = i-1
        

guillaume budé
françois ii de rohan
antoine de montbel
richelieu
louis xiii
nicolas fouquet
charles le brun
molière
madame de sévigné
charles perrault
françois de nesmond
madame de la fayette
nicolas boileau
louvois
louis xiv
marc
antoine charpentier
romain coupier de maille
jean de la bruyère
jules hardouin
mansart
françois couperin
saint
simon
voltaire
jean d alembert
madame de pompadour
louis xv
marquis de sade
jean
rené asseline
jacques
louis david
charles
maurice de talleyrand
périgord
françois
jean baudouin
charles percier
julie philipault
hippolyte ganneron
achille chaper
baptiste corot
jules michelet
littré
alexandre
gabriel decamps
adolphe adam
prosper mérimée
eugène sue
george sand
alexis de tocqueville
napoléon iii
gérard de nerval
baron haussmann
alfred de musset
viollet-le-duc
eugène labiche
eugène pottier
charles gounod
charles baudelaire
charles garnier
victor frédéric verrimst
gustave moreau
lise cristiani
jules de goncourt
édouard manet
edgar degas
camille saint
saëns


except - try again
michel cymes
mgr luc ravel
fred vargas
except - try again
françois asselineau
except - try again
christophe dechavanne
except - try again
laurent boyer
daniel schneidermann
laurent baffie
isabelle mergault
françois feldman
pierre sled
bernard thibault
pascal légitimus
patrick dupond
except - try again
luc besson
yasmina reza
rémy sarrazin
thierry marx
marc alexandre
antoine duléry
except - try again
harlem désir
bruno madinier
laetitia meignan
isabelle martinet
laurent fignon
françois zygel
frédéric taddeï
virginie lemoine
nicolas dupont
aignan
axel bauer
samy naceri
except - try again
marina carrère d encausse
catherine falgayrac
except - try again
gilles bouleau
claude pietragalla
pierre ménès
except - try again
alexandre pesle
élie semoun
philippe lelièvre
charly nestor
juliette binoche
except - try again
luc delarue
bruno solo
vincent moscato
anne roumanoff
olivier martinez
richard orlinski
josé garcia
guillaume de tonquédec
except - try again
hervé mathoux
sophi

In [68]:
df = pd.DataFrame(persos)
df

,nom,dateOfBirth,placeOfBirthLabel,dateOfDeath,placeOfDeathLabel,positions
0,Guillaume Budé,1467-02-04T00:00:00Z,Paris,1540-09-02T00:00:00Z,Paris,"prévôt des marchands de Paris, Maître de la Librairie du Roi"
1,François II de Rohan,1480-01-01T00:00:00Z,NaN,1536-10-23T00:00:00Z,Paris,archevêque catholique
2,Armand Jean du Plessis de Richelieu,1585-09-09T00:00:00Z,Paris,1642-12-04T00:00:00Z,Paris,"cardinal, évêque catholique, évêque diocésain, Secrétaire d'État des Affaires étrangères, gouverneur de Bretagne"
3,Louis XIII,1601-09-27T00:00:00Z,Fontainebleau,1643-05-14T00:00:00Z,Saint-Germain-en-Laye,roi de France et de Navarre
4,Nicolas Fouquet,1615-01-27T00:00:00Z,Paris,1680-03-23T00:00:00Z,Pignerol,"Surintendant des finances, Intendant de la généralité de Paris, Procureur général au Parlement de Paris, Intendant de la généralité de Grenoble"
5,Charles Le Brun,1619-02-24T00:00:00Z,Paris,1690-02-12T00:00:00Z,Paris,Premier peintre du Roi
6,Molière,1622-01-15T00:00:00Z,Paris,1673-02-17T00:00:00Z,Paris,
7,Marquise de Sévigné,1626-02-05T00:00:00Z,Paris,1696-04-17T00:00:00Z,Grignan,
8,Charles Perrault,1628-01-12T00:00:00Z,Paris,1703-05-16T00:00:00Z,Paris,fauteuil 23 de l'Académie française
9,François de Nesmond,1629-08-31T00:00:00Z,Paris,1715-06-16T00:00:00Z,Bayeux,"évêque catholique, évêque diocésain"


In [69]:
df.shape

(736, 6)

In [71]:
df.to_csv("personnages-paris-wiki.csv", index=False)

In [44]:
from SPARQLWrapper import SPARQLWrapper, JSON
import numpy as np
import pandas as pd
import folium
from folium.plugins import MarkerCluster